# ParaView Volume Rendering with NVIDIA IndeX

https://developer.nvidia.com/nvidia-index

In [ ]:
from paraview.simple import *
from paraview.modules.vtkRemotingCore import vtkProcessModule
LoadPlugin("/apps/daint/UES/jenkins/7.0.UP02-20.11/gpu/easybuild/software/ParaView/5.9.0-CrayGNU-20.11-EGL-python3/lib64/paraview-5.9/plugins/pvNVIDIAIndeX/pvNVIDIAIndeX.so", ns=globals())

In [ ]:
gs = GetSettingsProxy("RenderViewSettings")
gs.SetPropertyWithName("DisableIceT", 1)

info = GetOpenGLInformation(location=servermanager.vtkSMSession.RENDER_SERVER)

In [ ]:
Version = str(GetParaViewVersion())

print("ParaView Version ", Version)
print("Vendor:   %s" % info.GetVendor())
print("Version:  %s" % info.GetVersion())
print("Renderer: %s" % info.GetRenderer())

# NVIDIA IndeX Volume renderingis only available with NVIDIA hardware
assert info.GetVendor().split()[0] == 'NVIDIA'

In [ ]:
reader = ImageReader(FileNames=['/scratch/snx3000/jfavre/NVIDIA/50.density-log_1024x1024x2048_float.raw'])
reader.DataScalarType = 'float'
reader.DataByteOrder = 'LittleEndian'
reader.DataExtent = [0, 2047, 0, 1023, 0, 1023]

In [ ]:
renderView = CreateRenderView()
renderView.ViewSize = [600,600]
rep = Show(reader, GetActiveView())
rep.Representation = 'Outline'
rep.ColorArrayName = ['POINTS', 'ImageFile']

ImageFileLUT = GetColorTransferFunction('ImageFile')
ImageFileLUT.RGBPoints = [1.206409215927124, 0.231373, 0.298039, 0.752941, 5.729784369468689, 0.865003, 0.865003, 0.865003, 10.253159523010254, 0.705882, 0.0156863,
 0.14902]
ImageFileLUT.ScalarRangeInitialized = 1.0

# get opacity transfer function/opacity map for 'MetaImage'
ImageFilePWF = GetOpacityTransferFunction('ImageFile')
ImageFilePWF.Points = [1.206409215927124, 0.0, 0.5, 0.0, 10.253159523010254, 1.0, 0.5, 0.0]
ImageFilePWF.ScalarRangeInitialized = 1

# Here I recommend to wait until the cell has really finished to run.
# recall that we are running in parallel on the node and some extra sync does not hurt.

In [ ]:
from ipyparaview.widgets import PVDisplay
pvdisp = PVDisplay(GetActiveView())
w = display(pvdisp)

In [ ]:
def SaveImage(filename):
  from vtk import vtkPNGWriter
  img_writer = vtkPNGWriter()
  img_writer.SetInputConnection(pvdisp.w2i.GetOutputPort())
  img_writer.SetFileName(filename)
  img_writer.Write()

# save animage on the client-side
#SaveImage("/users/jfavre/foo2.png")

In [ ]:
rep.Representation = 'Volume'
rep.LookupTable = ImageFileLUT
rep.OpacityArray = ['POINTS', 'ImageFile']
rep.OpacityTransferFunction = 'PiecewiseFunction'
rep.ScalarOpacityUnitDistance = 4
rep.ScalarOpacityFunction = ImageFilePWF

In [ ]:
# get a list of available Representation modes
rep.Representation.Available

In [ ]:
rep.Representation = 'NVIDIA IndeX'

In [ ]:
ImageFileLUT = GetColorTransferFunction('ImageFile')
ImageFileLUT.RGBPoints = [1.206409215927124, 0.870485, 0.913768, 0.832905, 2.348207448379993, 0.31583, 0.776442, 0.867858, 3.552619695663452, 0.117909, 0.484134, 0.713825, 4.496826171875, 0.0, 0.209874, 0.511832, 5.297971248626709, 0.0157519, 0.00332021, 4.55569e-08, 6.814424514770508, 0.520865, 0.0, 0.0, 8.057207709363222, 0.785109, 0.339479, 0.000797922, 9.199010491175532, 0.910974, 0.699774, 0.0, 10.305128097534178, 0.881371, 0.912178, 0.818099]
ImageFileLUT.ColorSpace = 'Lab'
ImageFileLUT.ScalarRangeInitialized = 1.0

# get opacity transfer function/opacity map for 'ImageFile'
ImageFilePWF = GetOpacityTransferFunction('ImageFile')
ImageFilePWF.Points = [1.206409215927124, 0.30882352590560913, 0.5, 0.0, 2.293677568435669, 0.05147058889269829, 0.5, 0.0, 3.037597894668579, 0.029411764815449715, 0.5, 0.0, 3.4381704330444336, 0.0, 0.5, 0.0, 3.838742971420288, 0.0, 0.5, 0.0, 4.267927646636963, 0.0, 0.5, 0.0, 4.782949447631836, 0.07352941483259201, 0.5, 0.0, 5.069072723388672, 0.3014705777168274, 0.5, 0.0, 5.669931411743164, 0.5955882668495178, 0.5, 0.0, 7.014710903167725, 0.8161764740943909, 0.5, 0.0, 8.531164169311523, 0.9338235259056091, 0.5, 0.0, 10.305128097534178, 1.0, 0.5, 0.0]
ImageFilePWF.ScalarRangeInitialized = 1

rep.Filter = 'Gradient'
rep.gradient_scale = 120.0
rep.ScalarOpacityUnitDistance = 46.